<a href="https://colab.research.google.com/github/Neoken55/my-react-project/blob/main/Procon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
import numpy as np
import librosa
from scipy.io import wavfile
import scipy.io

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO     
!pip -q install pydub
from pydub import AudioSegment

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  audio = AudioSegment.from_file(BytesIO(b))
  return audio

In [7]:
# a = record(sec=2)
b = np.array(a.get_array_of_samples(), np.float32)
c = librosa.feature.mfcc(b)
print(a)
print(b)
b


[     0.      0.      0. ... -24202. -35617. -32787.]


array([     0.,      0.,      0., ..., -24202., -35617., -32787.],
      dtype=float32)

In [11]:
samplerate, data = wavfile.read("E01.wav")
x = data.astype('float32')
x = librosa.feature.mfcc(x)
x.shape

(20, 527)

In [ ]:
X = []
Y = []
names = ['left', 'right']

In [ ]:
def record_to_mfcc():
  print('start')
  x = record(sec=2)
  x = np.array(x.get_array_of_samples(), np.float32)
  return librosa.feature.mfcc(x)

In [ ]:
for i in range(10):
  x = record_to_mfcc()
  print(x.shape)
  X.append(x)
  Y.append(names.index('right'))

start


<IPython.core.display.Javascript object>

(20, 186)
start


<IPython.core.display.Javascript object>

(20, 186)
start


...

(20, 186)
start


<IPython.core.display.Javascript object>

(20, 186)
start


<IPython.core.display.Javascript object>

(20, 186)


In [ ]:
X = np.array(X)
Y = np.array(Y)

In [ ]:
X.shape

(20, 20, 186)

In [ ]:
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(64, 5, input_shape=(20, 186, 1), activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(64, 5, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X[:,:,:,None], Y, epochs=20)

Epoch 1/20
1/1 [==============================] - 1s 757ms/step - loss: 36.5444 - accuracy: 0.4500
Epoch 2/20
1/1 [==============================] - 0s 15ms/step - loss: 185.2634 - accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 13ms/step - loss: 36.0343 - accuracy: 0.5500
Epoch 4/20
1/1 [==============================] - 0s 11ms/step - loss: 54.4842 - accuracy: 0.5500
Epoch 5/20
1/1 [==============================] - 0s 11ms/step - loss: 30.0710 - accuracy: 0.5000
Epoch 6/20
1/1 [==============================] - 0s 10ms/step - loss: 21.1583 - accuracy: 0.6500
Epoch 7/20
1/1 [==============================] - 0s 12ms/step - loss: 22.8686 - accuracy: 0.7000
Epoch 8/20
1/1 [==============================] - 0s 10ms/step - loss: 21.3225 - accuracy: 0.6000
Epoch 9/20
1/1 [==============================] - 0s 10ms/step - loss: 12.2959 - accuracy: 0.4500
Epoch 10/20
1/1 [==============================] - 0s 12ms/step - loss: 12.0150 - accuracy: 0.5000
Epoch 11/20
1/1 [

In [ ]:
x = record_to_mfcc()
names[model.predict(x[None,:,:,None]).argmax()]

start


<IPython.core.display.Javascript object>

'right'